In [8]:
! pip install 'git+https://github.com/nitkiew2/CMSE495-QSIDE-JUSTFAIR-TOOLBOX.git'

  Cloning https://github.com/nitkiew2/CMSE495-QSIDE-JUSTFAIR-TOOLBOX.git to /private/var/folders/lm/qgrqsck97r52_d1s83h76nkc0000gn/T/pip-req-build-9_8kq0eh
  Running command git clone -q https://github.com/nitkiew2/CMSE495-QSIDE-JUSTFAIR-TOOLBOX.git /private/var/folders/lm/qgrqsck97r52_d1s83h76nkc0000gn/T/pip-req-build-9_8kq0eh
  Resolved https://github.com/nitkiew2/CMSE495-QSIDE-JUSTFAIR-TOOLBOX.git to commit ebd58f3433db6843d1cb819248823010a183ec94
    ERROR: Command errored out with exit status 1:
     command: /Users/jason/opt/anaconda3/bin/python -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folders/lm/qgrqsck97r52_d1s83h76nkc0000gn/T/pip-req-build-9_8kq0eh/setup.py'"'"'; __file__='"'"'/private/var/folders/lm/qgrqsck97r52_d1s83h76nkc0000gn/T/pip-req-build-9_8kq0eh/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"

In [3]:
import JUSTFAIR_Tools as jt